### Desafio Inferência - Depressão e Habitos Saudáveis
O transtorno depressivo representa um grande problema de saúde pública e é apontado como uma das principais causas de doenças e debilitações segundo a Organização Mundial da Saúde (OMS). A depressão resulta de uma interação complexa de fatores sociais, psicológicos e biológicos e, embora o acesso a diagnósticos e tratamentos esteja aumentando, muitos ainda não possuem acesso ao controle adequado dos sintomas depressivos.

Para garantir um estilo de vida saudável, a OMS recomenda a prática regular de exercícios, bem como a adoção de uma dieta saudável rica em alimentos in natura e com reduzido teor de gordura saturada, sal e açúcares refinados.

Neste desafio, utilizaremos dados de uma pesquisa realizada anualmente nos Estados Unidos para avaliar (1) qual o perfil de indivíduos (adultos maiores de 18 anos) com sintomas depressivos nos EUA no período de 2005-2006, e (2) se hábitos saudáveis de alimentação e atividade física estão associados a menores índices de depressão nesta população.


## Importando Bibliotecas

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pingouin as pg
from pingouin import pairwise_corr
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pingouin as pg
import random 
import scipy.stats
from scipy import stats
import janitor

## Definindo Padrão dos Gráficos

In [3]:
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (12, 8),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

%matplotlib inline
plt.rcParams.update(params)

## Leitura dos Arquivos

In [6]:
phq = pd.read_csv(r"DEMO_PHQ.csv",
                sep=',',
                engine='python').clean_names()

In [5]:
hei = pd.read_csv(r"PAG_HEI.csv",
                sep=',',
                engine='python').clean_names()

In [7]:
phq.head()


,seqn,dpq010,dpq020,dpq030,dpq040,dpq050,dpq060,dpq070,dpq080,dpq090,riagendr,ridageyr,ridreth1,dmdeduc,indfminc
0,31130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,85,3,4,4.0
1,31131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,44,4,4,11.0
2,31132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,70,3,5,11.0
3,31134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,73,3,3,12.0
4,31139,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,2,18,2,3,11.0


In [9]:
hei.head()

,seqn,pag_minw,adherence,hei2015c1_totalveg,hei2015c2_green_and_bean,hei2015c3_totalfruit,hei2015c4_wholefruit,hei2015c5_wholegrain,hei2015c6_totaldairy,hei2015c7_totprot,hei2015c8_seaplant_prot,hei2015c9_fattyacid,hei2015c10_sodium,hei2015c11_refinedgrain,hei2015c12_sfat,hei2015c13_addsug,hei2015_total_score
0,31128,NaN,NaN,3.083350,0.0,3.927008,5.000000,0.000000,5.000000,4.251329,0.000000,2.487572,6.911882,10.000000,0.818029,6.408092,47.887262
1,31129,285.450346,2.0,1.697761,0.0,3.731602,4.929186,0.527631,3.887809,4.693141,0.928562,3.740758,6.146441,7.274979,8.006457,0.000000,45.564326
2,31130,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31131,264.711316,2.0,3.115103,0.0,5.000000,1.320674,0.000000,3.141062,4.071959,2.052399,0.684985,2.297100,9.512964,0.438428,3.919226,35.553899
4,31132,1353.348730,3.0,3.509081,0.0,5.000000,5.000000,5.000000,4.448512,5.000000,0.000000,2.050769,7.369489,8.887880,8.186877,9.099088,63.551696


In [10]:
## Consultando valores distintoss

#hei.seqn.nunique()
phq.seqn.nunique()

5334

## Mesclando Datasets

In [13]:
df = pd.merge(
        phq.reset_index(), 
        hei.reset_index(),
        on=['seqn']
        )

,index_x,seqn,dpq010,dpq020,dpq030,dpq040,dpq050,dpq060,dpq070,dpq080,...,hei2015c5_wholegrain,hei2015c6_totaldairy,hei2015c7_totprot,hei2015c8_seaplant_prot,hei2015c9_fattyacid,hei2015c10_sodium,hei2015c11_refinedgrain,hei2015c12_sfat,hei2015c13_addsug,hei2015_total_score
0,0,31130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,31131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,3.141062,4.071959,2.052399,0.684985,2.297100,9.512964,0.438428,3.919226,35.553899
2,2,31132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.000000,4.448512,5.000000,0.000000,2.050769,7.369489,8.887880,8.186877,9.099088,63.551696
3,3,31134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.217331,2.313900,5.000000,1.268648,2.958985,0.000000,5.627488,5.863078,5.878686,42.886826
4,4,31139,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,...,0.211544,0.000000,4.088849,4.221442,10.000000,2.845774,0.000000,10.000000,9.658507,47.074393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5329,5329,41466,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,4.607930,2.555736,5.000000,4.672204,4.633477,0.000000,6.580006,8.393244,9.360990,59.949915
5330,5330,41468,0.0,2.0,0.0,1.0,1.0,2.0,1.0,3.0,...,3.041603,5.000000,3.759596,2.344264,4.780282,3.134216,1.986137,5.850269,10.000000,57.747869
5331,5331,41469,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,2.151846,5.000000,0.000000,9.344947,3.535211,4.326307,5.269780,8.195564,42.993323
5332,5332,41472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.933303,5.000000,5.000000,5.000000,2.090534,8.409844,8.936854,3.063795,0.000000,46.134467
